# Data exploration

Here I take an initial look at the available data. I will only include a subset of the rows of the total data to get an idea about the contents. This way we don't need a Spark cluster but can simply use pandas on a local machine.

In [2]:
import json
import pandas as pd
import s3fs
import config
config.configure()

## Airport data

The airport data is in JSON format. Let's load it and have a look:

In [3]:
airports = pd.read_json('s3a://atr-udacity-dend/airport-codes.json')
airports

,continent,coordinates,elevation_ft,gps_code,iata_code,ident,iso_country,iso_region,local_code,municipality,name,type
0,NA,"-74.93360137939453, 40.07080078125",11.0,00A,None,00A,US,US-PA,00A,Bensalem,Total Rf Heliport,heliport
1,NA,"-101.473911, 38.704022",3435.0,00AA,None,00AA,US,US-KS,00AA,Leoti,Aero B Ranch Airport,small_airport
2,NA,"-151.695999146, 59.94919968",450.0,00AK,None,00AK,US,US-AK,00AK,Anchor Point,Lowell Field,small_airport
3,NA,"-86.77030181884766, 34.86479949951172",820.0,00AL,None,00AL,US,US-AL,00AL,Harvest,Epps Airpark,small_airport
4,NA,"-91.254898, 35.6087",237.0,None,None,00AR,US,US-AR,None,Newport,Newport Hospital & Clinic Heliport,closed
...,...,...,...,...,...,...,...,...,...,...,...,...
57416,AS,"122.3586, 40.542524",0.0,ZYYK,YKH,ZYYK,CN,CN-21,None,Yingkou,Yingkou Lanqi Airport,medium_airport
57417,AS,"123.49600219726562, 41.784400939941406",NaN,ZYYY,None,ZYYY,CN,CN-21,None,Shenyang,Shenyang Dongta Airport,medium_airport
57418,EU,"1.4825, 51.894444",40.0,None,None,ZZ-0001,GB,GB-ENG,None,Sealand,Sealand Helipad,heliport
57419,AF,"47.296388888900005, -11.584277777799999",11.0,None,None,ZZ-0002,TF,TF-U-A,None,Grande Glorieuse,Glorioso Islands Airstrip,small_airport


The dataset contains approx. 57,000 airports. The airports are identified with an "ident" code, and some airports also have an international 3-letter IATA code. Airports are categorized into heliport, small_airport, medium_airport etc. Additionally airports have a name and location data such as country, region, municipality, coordinates and elevation in feet.

Looking closer at the columns reveals some columns are missing data, specifically elevation_ft, gps_code, iata_code, local_code, and municipality:

In [4]:
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57421 entries, 0 to 57420
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   continent     57421 non-null  object 
 1   coordinates   57421 non-null  object 
 2   elevation_ft  49608 non-null  float64
 3   gps_code      41561 non-null  object 
 4   iata_code     9225 non-null   object 
 5   ident         57421 non-null  object 
 6   iso_country   57421 non-null  object 
 7   iso_region    57421 non-null  object 
 8   local_code    30030 non-null  object 
 9   municipality  51527 non-null  object 
 10  name          57421 non-null  object 
 11  type          57421 non-null  object 
dtypes: float64(1), object(11)
memory usage: 5.3+ MB


The ident column has no duplicates:

In [5]:
airports['ident'].nunique()

57421

The iata_code column does have some duplicates:

In [6]:
iata_code_counts = airports['iata_code'].value_counts()
iata_code_counts[iata_code_counts > 1]

0      75
MPT     2
YNT     2
TFY     2
SVD     2
KCZ     2
MRE     2
VQS     2
RTI     2
SCR     2
HKG     2
ZRZ     2
DDU     2
PHM     2
RZS     2
NWT     2
PRM     2
ARX     2
GGC     2
MUC     2
MNI     2
YEH     2
DZI     2
KMM     2
PCO     2
ESP     2
SGL     2
LLJ     2
KWB     2
LHU     2
YTY     2
SHO     2
LPE     2
DLR     2
MXR     2
LMC     2
IZA     2
CLG     2
AHT     2
Name: iata_code, dtype: int64

Looking at some of the duplicates does not reveal a simple reason, so I will have to deal with this inconsistency by simply ignoring one of the rows.

In [7]:
airports[airports['iata_code'] == 'MPT']

,continent,coordinates,elevation_ft,gps_code,iata_code,ident,iso_country,iso_region,local_code,municipality,name,type
24774,AS,"125.00900268554688, -8.167739868164062",957.0,None,MPT,ID-MPT,ID,ID-NT,None,Maliana-Alor Island,Maliana airport,small_airport
54534,AS,"125.214996, -8.97224",NaN,WPMN,MPT,WPMN,TL,TL-BO,None,Maliana,Maliana Airport,small_airport


In [8]:
airports[airports['iata_code'] == 'SCR']

,continent,coordinates,elevation_ft,gps_code,iata_code,ident,iso_country,iso_region,local_code,municipality,name,type
43233,NA,"-75.772, 41.4802",1151.0,None,SCR,SCR,US,US-PA,None,Scranton,Scranton Municipal Airport.,closed
44104,EU,"12.840676, 61.159092",1608.0,ESKS,SCR,SE-0016,SE,SE-W,None,SÃ¤len and Trysil,Scandinavian Mountains Airport,medium_airport


For this project I am interested in U.S. airports, having an IATA code assigned. This code will be used later to look up airport data for flights.

Let's reduce the dataset to US airports with an IATA code, and drop duplicates:

In [9]:
us_iata_airports = airports[airports['iso_country'] == 'US'].dropna(subset=['iata_code']).drop_duplicates(subset=['iata_code'])
us_iata_airports

,continent,coordinates,elevation_ft,gps_code,iata_code,ident,iso_country,iso_region,local_code,municipality,name,type
441,NA,"-80.274803161621, 25.325399398804",8.0,07FA,OCA,07FA,US,US-FL,07FA,Key Largo,Ocean Reef Club Airport,small_airport
598,NA,"-162.899994, 61.934601",305.0,None,PQS,0AK,US,US-AK,0AK,Pilot Station,Pilot Station Airport,small_airport
677,NA,"-106.928341, 38.851918",8980.0,0CO2,CSE,0CO2,US,US-CO,0CO2,Crested Butte,Crested Butte Airpark,small_airport
1094,NA,"-98.62249755859999, 30.251800537100003",1515.0,0TE7,JCY,0TE7,US,US-TX,0TE7,Johnson City,LBJ Ranch Airport,small_airport
1408,NA,"-72.31140136719999, 42.223300933800004",418.0,13MA,PMX,13MA,US,US-MA,13MA,Palmer,Metropolitan Airport,small_airport
...,...,...,...,...,...,...,...,...,...,...,...,...
56907,NA,"-133.79699707, 55.9663009644",0.0,Z71,CZP,Z71,US,US-AK,Z71,Cape Pole,Cape Pole Seaplane Base,seaplane_base
56908,NA,"-158.401000977, 56.295600891099994",NaN,Z78,KBW,Z78,US,US-AK,Z78,Chignik,Chignik Bay Seaplane Base,seaplane_base
56913,NA,"-145.824005127, 66.2740020752",450.0,Z91,KBC,Z91,US,US-AK,Z91,Birch Creek,Birch Creek Airport,small_airport
56915,NA,"-145.294006348, 61.9412002563",1150.0,Z93,CZC,Z93,US,US-AK,Z93,Copper Center,Copper Center 2 Airport,small_airport


Around 2,000 airports remain, and the airport types are as follows:

In [10]:
us_iata_airports['type'].unique()

array(['small_airport', 'seaplane_base', 'closed', 'medium_airport',
       'heliport', 'large_airport'], dtype=object)

## Flight data

The flight data is in CSV format, with one CSV file per month for the year of 2019. In the data pipeline I will use all of the data, but here let's just have a look at a a few of the months:

In [27]:
flights = pd.concat((pd.read_csv(f"s3a://atr-udacity-dend/flightdata/2019/{month}.csv", dtype=str) for month in ["01", "02", "03"]))
flights

,FL_DATE,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,DEST,DEST_CITY_NAME,DEST_STATE_ABR,CRS_DEP_TIME,DEP_TIME,TAXI_OUT,...,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2019-01-01,PIT,"Pittsburgh, PA",PA,CLT,"Charlotte, NC",NC,0701,0657,14.0,...,86.0,67.0,1.0,366.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,DFW,"Dallas/Fort Worth, TX",TX,SEA,"Seattle, WA",WA,0910,0948,13.0,...,250.0,225.0,1.0,1660.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,SEA,"Seattle, WA",WA,DFW,"Dallas/Fort Worth, TX",TX,1250,1250,13.0,...,264.0,201.0,1.0,1660.0,0.0,0.0,24.0,0.0,0.0,NaN
3,2019-01-01,MCO,"Orlando, FL",FL,PHL,"Philadelphia, PA",PA,1632,1620,14.0,...,135.0,116.0,1.0,861.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,PHL,"Philadelphia, PA",PA,MCO,"Orlando, FL",FL,1245,1239,11.0,...,152.0,129.0,1.0,861.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
632069,2019-03-31,DCA,"Washington, DC",VA,MHT,"Manchester, NH",NH,1505,1459,18.0,...,87.0,63.0,1.0,406.0,NaN,NaN,NaN,NaN,NaN,NaN
632070,2019-03-31,MHT,"Manchester, NH",NH,DCA,"Washington, DC",VA,1717,1709,13.0,...,138.0,120.0,1.0,406.0,0.0,0.0,27.0,0.0,0.0,NaN
632071,2019-03-31,DCA,"Washington, DC",VA,RDU,"Raleigh/Durham, NC",NC,0700,0656,11.0,...,61.0,44.0,1.0,227.0,NaN,NaN,NaN,NaN,NaN,NaN
632072,2019-03-31,PHL,"Philadelphia, PA",PA,BNA,"Nashville, TN",TN,1245,1237,11.0,...,127.0,109.0,1.0,675.0,NaN,NaN,NaN,NaN,NaN,NaN


There are too many columns to print out, so let's look at all columns just for the first row of data:

In [28]:
flights.iloc[0]

FL_DATE                    2019-01-01
ORIGIN                            PIT
ORIGIN_CITY_NAME       Pittsburgh, PA
ORIGIN_STATE_ABR                   PA
DEST                              CLT
DEST_CITY_NAME          Charlotte, NC
DEST_STATE_ABR                     NC
CRS_DEP_TIME                     0701
DEP_TIME                         0657
TAXI_OUT                         14.0
WHEELS_OFF                       0711
WHEELS_ON                        0818
TAXI_IN                           5.0
CRS_ARR_TIME                     0849
ARR_TIME                         0823
CANCELLED                         0.0
CANCELLATION_CODE                 NaN
CRS_ELAPSED_TIME                108.0
ACTUAL_ELAPSED_TIME              86.0
AIR_TIME                         67.0
FLIGHTS                           1.0
DISTANCE                        366.0
CARRIER_DELAY                     NaN
WEATHER_DELAY                     NaN
NAS_DELAY                         NaN
SECURITY_DELAY                    NaN
LATE_AIRCRAF

The data contains a flight date, origin and destination airports identified by the 3-letter international IATA code, as well as several timestamps and duration columns. Timestamps are encoded as four digits and identify scheduled and actual departure/arrival times, and wheels on/off times. Durations encoded as number of minutes measure taxi time and delays.

Looking closer at the columns shows the origin/destination data is always available, the planned departure/arrival times are available as well, but the actual times and delays are not always present:

In [29]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1749234 entries, 0 to 632073
Data columns (total 28 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   FL_DATE              object
 1   ORIGIN               object
 2   ORIGIN_CITY_NAME     object
 3   ORIGIN_STATE_ABR     object
 4   DEST                 object
 5   DEST_CITY_NAME       object
 6   DEST_STATE_ABR       object
 7   CRS_DEP_TIME         object
 8   DEP_TIME             object
 9   TAXI_OUT             object
 10  WHEELS_OFF           object
 11  WHEELS_ON            object
 12  TAXI_IN              object
 13  CRS_ARR_TIME         object
 14  ARR_TIME             object
 15  CANCELLED            object
 16  CANCELLATION_CODE    object
 17  CRS_ELAPSED_TIME     object
 18  ACTUAL_ELAPSED_TIME  object
 19  AIR_TIME             object
 20  FLIGHTS              object
 21  DISTANCE             object
 22  CARRIER_DELAY        object
 23  WEATHER_DELAY        object
 24  NAS_DELAY            obje

Here is an example of a delayed flight, in this case delayed 24 minutes caused by the National Air System (NAS_DELAY):

In [30]:
flights[flights['ACTUAL_ELAPSED_TIME'] > flights['CRS_ELAPSED_TIME']].iloc[0]

FL_DATE                    2019-01-01
ORIGIN                            PIT
ORIGIN_CITY_NAME       Pittsburgh, PA
ORIGIN_STATE_ABR                   PA
DEST                              CLT
DEST_CITY_NAME          Charlotte, NC
DEST_STATE_ABR                     NC
CRS_DEP_TIME                     0701
DEP_TIME                         0657
TAXI_OUT                         14.0
WHEELS_OFF                       0711
WHEELS_ON                        0818
TAXI_IN                           5.0
CRS_ARR_TIME                     0849
ARR_TIME                         0823
CANCELLED                         0.0
CANCELLATION_CODE                 NaN
CRS_ELAPSED_TIME                108.0
ACTUAL_ELAPSED_TIME              86.0
AIR_TIME                         67.0
FLIGHTS                           1.0
DISTANCE                        366.0
CARRIER_DELAY                     NaN
WEATHER_DELAY                     NaN
NAS_DELAY                         NaN
SECURITY_DELAY                    NaN
LATE_AIRCRAF

Here is an example of a cancelled flight with cancellation code B (weather):

In [31]:
flights[flights['CANCELLED'] == "1.00"].iloc[0]

FL_DATE                   2019-01-01
ORIGIN                           CLT
ORIGIN_CITY_NAME       Charlotte, NC
ORIGIN_STATE_ABR                  NC
DEST                             PNS
DEST_CITY_NAME         Pensacola, FL
DEST_STATE_ABR                    FL
CRS_DEP_TIME                    1955
DEP_TIME                        1953
TAXI_OUT                        15.0
WHEELS_OFF                      2008
WHEELS_ON                        NaN
TAXI_IN                          NaN
CRS_ARR_TIME                    2044
ARR_TIME                         NaN
CANCELLED                        1.0
CANCELLATION_CODE                  B
CRS_ELAPSED_TIME               109.0
ACTUAL_ELAPSED_TIME              NaN
AIR_TIME                         NaN
FLIGHTS                          1.0
DISTANCE                       488.0
CARRIER_DELAY                    NaN
WEATHER_DELAY                    NaN
NAS_DELAY                        NaN
SECURITY_DELAY                   NaN
LATE_AIRCRAFT_DELAY              NaN
U